In [1]:
# ========================================
# 1) Connect to Google Drive
# ========================================
from google.colab import drive
drive.mount('/content/drive')

MODEL_DIR = "/content/drive/MyDrive/deep_learning"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# ========================================
# 2) Install required libraries
# ========================================
!pip install -U transformers datasets accelerate evaluate optuna wandb


In [3]:
# ========================================
# 3) Import libraries
# ========================================
import os
import numpy as np
import pandas as pd
from datasets import Dataset, DatasetDict
from collections import Counter

import torch
from torch import nn

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
import evaluate
import wandb
import types
import torch

In [4]:
# Check if GPU is available and select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
# ========================================
# 4) Setup Weights & Biases logging
# ========================================
os.environ["WANDB_PROJECT"] = "covid-HF-1"
os.environ["WANDB_WATCH"] = "all"
os.environ["WANDB_LOG_MODEL"] = "true"
wandb.login()


wandb: Currently logged in as: yardenr1 (yardenr1-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

f423cfbdb50d571d7922fe745356685617100809

In [6]:
# ========================================
# 3) Load data (your preprocessed CSVs)
# ========================================
train_df = pd.read_csv('/content/drive/MyDrive/deep_learning/train_processed.csv', encoding='latin1')
eval_df  = pd.read_csv('/content/drive/MyDrive/deep_learning/val_processed.csv',   encoding='latin1')
test_df  = pd.read_csv('/content/drive/MyDrive/deep_learning/test_processed.csv',  encoding='latin1')

# Create label mappings
ordered_labels = ['Extremely Negative', 'Negative', 'Neutral', 'Positive', 'Extremely Positive']
label2id = {label: i for i, label in enumerate(ordered_labels)}
id2label = {i: label for label, i in label2id.items()}

train_df["label"] = train_df["Sentiment"].map(label2id)
eval_df["label"] = eval_df["Sentiment"].map(label2id)
test_df["label"] = test_df["Sentiment"].map(label2id)

for light cleaning:

In [8]:
import re
# ---------- 1) Dirty preprocessing from OriginalTweet ----------
def preprocess_tweet_dirty(t: str) -> str:
    if not isinstance(t, str):
        return ""
    t = re.sub(r'https?://\S+', 'HTTPURL', t)   # normalize urls
    t = re.sub(r'@\w+', '@USER', t)             # normalize users
    t = re.sub(r'\s+', ' ', t).strip()          # collapse spaces
    return t

_EMOJI_RE = re.compile(
    r'['
    r'\U0001F1E0-\U0001F1FF'  # flags
    r'\U0001F300-\U0001F5FF'  # symbols & pictographs
    r'\U0001F600-\U0001F64F'  # emoticons
    r'\U0001F680-\U0001F6FF'  # transport & map
    r'\U0001F700-\U0001F77F'
    r'\U0001F780-\U0001F7FF'
    r'\U0001F800-\U0001F8FF'
    r'\U0001F900-\U0001F9FF'
    r'\U0001FA00-\U0001FA6F'
    r'\U0001FA70-\U0001FAFF'
    r'\u2600-\u26FF\u2700-\u27BF'
    r']+'
)

def add_markers(text: str) -> str:
    text = re.sub(r'(?<!<HASHTAG>)#\w+', lambda m: f"<HASHTAG> {m.group(0)}", text)
    text = _EMOJI_RE.sub(lambda m: f"<EMOJI> {m.group(0)}", text)
    return text

# assume train_df, eval_df, test_df are already loaded + 'label' int exists
for df in (train_df, eval_df, test_df):
    df["ProcessedTweet"] = df["OriginalTweet"].apply(preprocess_tweet_dirty).apply(add_markers)


In [9]:
# ========================================
# 4) Build HF Datasets and tokenize
# ========================================
model_ckpt = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def tok(batch):
    return tokenizer(batch["ProcessedTweet"], truncation=True, padding="max_length", max_length=128)

ds = DatasetDict({
    "train": Dataset.from_pandas(train_df[["ProcessedTweet","label"]].rename(columns={"label":"labels"})),
    "validation": Dataset.from_pandas(eval_df[["ProcessedTweet","label"]].rename(columns={"label":"labels"})),
    "test": Dataset.from_pandas(test_df[["ProcessedTweet","label"]].rename(columns={"label":"labels"})),
})

ds_tok = ds.map(tok, batched=True)
ds_tok = ds_tok.remove_columns(["ProcessedTweet"])
ds_tok.set_format("torch")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/32925 [00:00<?, ? examples/s]

Map:   0%|          | 0/8232 [00:00<?, ? examples/s]

Map:   0%|          | 0/3798 [00:00<?, ? examples/s]

In [10]:
# ========================================
# 5) Optional: class weights (to handle class imbalance)
# ========================================
def compute_class_weights(int_labels, num_labels):
    """
    Compute inverse-frequency class weights normalized around 1.0
    """
    import numpy as np
    counts = np.bincount(int_labels, minlength=num_labels)
    weights = (counts.sum() / (counts + 1e-9)) / num_labels
    return weights / weights.mean()

class_weights = compute_class_weights(
    train_df["label"].to_numpy(),
    num_labels=len(ordered_labels)
)
class_weights
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

In [11]:
# ========================================
# 6) Custom RoBERTa with configurable dropout (CLS pooling)
# ========================================
import torch
import torch.nn as nn
from transformers import AutoModel

class RobertaWithDropout(nn.Module):
    """
    Custom RoBERTa head:
      - Takes CLS embedding (token 0)
      - Applies configurable dropout
      - Linear classifier to num_labels
    """
    def __init__(self, model_name, num_labels, dropout_rate=0.2):
        super().__init__()
        self.roberta = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.roberta.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        cls = outputs.last_hidden_state[:, 0, :]
        x = self.dropout(cls)
        logits = self.classifier(x)
        # Return dict-style compatible with HF Trainer
        return {"logits": logits}


In [12]:
# ========================================
# 7) Weighted loss wrapper for Trainer
# ========================================
import numpy as np
import torch
import torch.nn as nn

ce_loss = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float))

def custom_compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
    labels  = inputs.get("labels")
    outputs = model(
        input_ids=inputs["input_ids"],
        attention_mask=inputs.get("attention_mask")
    )
    logits = outputs["logits"] if isinstance(outputs, dict) else outputs.logits
    ce = nn.CrossEntropyLoss(weight=class_weights_tensor.to(logits.device))

    loss = ce(logits, labels)
    return (loss, outputs) if return_outputs else loss



In [13]:
# ========================================
# 8) Metrics (accuracy + F1 micro/macro/weighted)
# ========================================
import evaluate
acc = evaluate.load("accuracy")
f1  = evaluate.load("f1")

def compute_metrics_fn(eval_pred):
    """
    HF Trainer metrics: returns dict with accuracy and F1 variants
    """
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy":      acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1_micro":      f1.compute(predictions=preds, references=labels, average="micro")["f1"],
        "f1_macro":      f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "f1_weighted":   f1.compute(predictions=preds, references=labels, average="weighted")["f1"],
    }


In [14]:
# ========================================
# 9) Trainer factory (no HF checkpoints) + BestWeightsSaver callback
#    - Saves ONLY best weights (state_dict) per run to a temp .pt
#    - EarlyStopping relies on eval_f1_weighted from compute_metrics_fn
# ========================================
import os
import gc
import torch
import types
from datetime import datetime
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, TrainerCallback

class BestWeightsSaver(TrainerCallback):
    """
    Whenever 'eval_f1_weighted' improves, save ONLY model.state_dict() to `best_weights_path`.
    """
    def __init__(self, best_weights_path: str, metric_name: str = "eval_f1_weighted"):
        self.best_weights_path = best_weights_path
        self.metric_name = metric_name
        self.best_score = None
        self._trainer = None  # injected on attach

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if not metrics or self.metric_name not in metrics:
            return
        score = float(metrics[self.metric_name])
        if (self.best_score is None) or (score > self.best_score):
            self.best_score = score
            torch.save(self._trainer.model.state_dict(), self.best_weights_path)

MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment-latest"

def make_trainer(
    output_dir,
    dropout_rate=0.2,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.06,
    lr_scheduler_type="linear",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    gradient_accumulation_steps=1,
    label_smoothing_factor=0.0,
    fp16=True,
    report_to_wandb=False,
    run_name="trial",
    best_weights_path="/tmp/best_weights.pt",
):
    # Build model with your custom head
    model = RobertaWithDropout(
        model_name=MODEL_NAME,
        num_labels=len(ordered_labels),
        dropout_rate=dropout_rate
    )

    # Absolutely NO HF checkpoints to Drive
    args = TrainingArguments(
        output_dir=output_dir,                 # keep this under /tmp to avoid Drive writes
        eval_strategy="epoch",
        save_strategy="no",                    # no checkpoints
        load_best_model_at_end=False,          # we handle "best" ourselves
        metric_for_best_model="f1_weighted",   # required for EarlyStopping
        greater_is_better=True,
        logging_strategy="steps",
        logging_steps=50,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        num_train_epochs=num_train_epochs,
        warmup_ratio=warmup_ratio,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        lr_scheduler_type=lr_scheduler_type,
        gradient_accumulation_steps=gradient_accumulation_steps,
        label_smoothing_factor=label_smoothing_factor,
        max_grad_norm=1.0,
        fp16=fp16,
        report_to=(["wandb"] if report_to_wandb else ["none"]),
        run_name=run_name,
        seed=42,
        dataloader_num_workers=2,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=ds_tok["train"],
        eval_dataset=ds_tok["validation"],
        processing_class=tokenizer,           # future-proof vs tokenizer=
        compute_metrics=compute_metrics_fn,
    )

    # Custom weighted CE loss (device-aware)
    trainer.compute_loss = types.MethodType(custom_compute_loss, trainer)

    # Early stopping (small patience to keep it agile)
    trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))

    # Attach best-weights saver
    best_cb = BestWeightsSaver(best_weights_path=best_weights_path, metric_name="eval_f1_weighted")
    best_cb._trainer = trainer
    trainer.add_callback(best_cb)

    return trainer


In [15]:
# ========================================
# 10) Generic experiment runner (replaces your old "part 10")
#     - tune_once(): Optuna tuning for a given search space
#     - final_train_and_save(): long-ish final run + save ONLY weights to Drive
# ========================================
import json
import optuna
import pandas as pd

MODEL_DIR = "/content/drive/MyDrive/deep_learning"  # as you requested
EXPERIMENTS_LOG = os.path.join(MODEL_DIR, "HF_experiments_log_3.csv")  # append-only CSV

def now_tag():
    return datetime.now().strftime("%Y%m%d_%H%M%S")

def temp_paths(tag: str):
    stamp = now_tag()
    out_dir = f"/tmp/{tag}_{stamp}"
    best_pt = f"/tmp/{tag}_{stamp}_best.pt"
    os.makedirs(out_dir, exist_ok=True)
    return out_dir, best_pt

def append_row_to_log(row: dict):
    if os.path.exists(EXPERIMENTS_LOG):
        df = pd.read_csv(EXPERIMENTS_LOG)
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    else:
        df = pd.DataFrame([row])
    df.to_csv(EXPERIMENTS_LOG, index=False)

def tune_once(config_name: str, search_space_fn, n_trials=10, tune_epochs=(3,6)):
    """
    Run Optuna on a provided search space function.
    Returns (best_params, best_value).
    """
    def objective(trial: optuna.trial.Trial):
        hp = search_space_fn(trial, tune_epochs)
        out_dir, best_pt = temp_paths(f"tune_{config_name}_t{trial.number}")
        trainer = make_trainer(
            output_dir=out_dir,
            dropout_rate=hp["dropout_rate"],
            learning_rate=hp["learning_rate"],
            weight_decay=hp["weight_decay"],
            warmup_ratio=hp["warmup_ratio"],
            lr_scheduler_type=hp["lr_scheduler_type"],
            per_device_train_batch_size=hp["per_device_train_batch_size"],
            per_device_eval_batch_size=64,
            num_train_epochs=hp["num_train_epochs"],
            gradient_accumulation_steps=hp["gradient_accumulation_steps"],
            label_smoothing_factor=hp["label_smoothing_factor"],
            fp16=True,
            report_to_wandb=False,
            run_name=f"{config_name}-trial-{trial.number}",
            best_weights_path=best_pt
        )
        trainer.train()
        metrics = trainer.evaluate(ds_tok["validation"])
        score = float(metrics.get("eval_f1_weighted") or metrics.get("f1_weighted") or 0.0)

        # Clean temp
        try:
            if os.path.exists(best_pt):
                os.remove(best_pt)
        except OSError:
            pass
        del trainer
        torch.cuda.empty_cache(); gc.collect()
        return score

    study_name = f"{config_name}_{now_tag()}"
    study = optuna.create_study(direction="maximize", study_name=study_name)
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    # log summary
    row = {
        "time": now_tag(),
        "phase": "tuning",
        "config": config_name,
        "study_name": study_name,
        "best_value_f1w": study.best_value,
        "best_params_json": json.dumps(study.best_trial.params, ensure_ascii=False),
        "n_trials": n_trials,
        "tune_epochs": str(tune_epochs),
    }
    append_row_to_log(row)
    print(f"======= Tuning finished: {config_name} | best f1_weighted={study.best_value:.4f}")
    return study.best_trial.params, study.best_value

def final_train_and_save(config_name: str, best_params: dict, final_epochs=12, save_name=None):
    """
    Final training with best params (longer, ES active), save ONLY weights .pt to Drive,
    log val/test to CSV.
    """
    out_dir, best_pt = temp_paths(f"final_{config_name}")
    trainer = make_trainer(
        output_dir=out_dir,
        dropout_rate=best_params.get("dropout_rate", 0.2),
        learning_rate=best_params.get("learning_rate", 2e-5),
        weight_decay=best_params.get("weight_decay", 0.01),
        warmup_ratio=best_params.get("warmup_ratio", 0.06),
        lr_scheduler_type=best_params.get("lr_scheduler_type", "linear"),
        per_device_train_batch_size=best_params.get("per_device_train_batch_size", 32),
        per_device_eval_batch_size=64,
        num_train_epochs=final_epochs,
        gradient_accumulation_steps=best_params.get("gradient_accumulation_steps", 1),
        label_smoothing_factor=best_params.get("label_smoothing_factor", 0.0),
        fp16=True,
        report_to_wandb=False,
        run_name=f"{config_name}-final",
        best_weights_path=best_pt
    )
    trainer.train()
    val_metrics  = trainer.evaluate(ds_tok["validation"])
    test_metrics = trainer.evaluate(ds_tok["test"])

    # Rebuild and save ONLY weights to Drive
    model_for_save = RobertaWithDropout(
        model_name=MODEL_NAME,
        num_labels=len(ordered_labels),
        dropout_rate=best_params.get("dropout_rate", 0.2),
    )
    assert os.path.exists(best_pt), "Temp best weights not found."
    model_for_save.load_state_dict(torch.load(best_pt, map_location="cpu"))

    if save_name is None:
        save_name = f"HF_best_{config_name}_{now_tag()}.pt"
    drive_path = os.path.join(MODEL_DIR, save_name)
    torch.save(model_for_save.state_dict(), drive_path)
    print(f"========= Final best weights saved to: {drive_path}")

    # cleanup temp
    try:
        os.remove(best_pt)
    except OSError:
        pass
    del trainer, model_for_save
    torch.cuda.empty_cache(); gc.collect()

    # append final results
    row = {
        "time": now_tag(),
        "phase": "final",
        "config": config_name,
        "val_f1_weighted": float(val_metrics.get("eval_f1_weighted", 0.0)),
        "val_accuracy": float(val_metrics.get("eval_accuracy", 0.0)),
        "test_f1_weighted": float(test_metrics.get("eval_f1_weighted", 0.0)),
        "test_accuracy": float(test_metrics.get("eval_accuracy", 0.0)),
        "saved_to": drive_path,
    }
    append_row_to_log(row)
    return drive_path, val_metrics, test_metrics


In [ ]:
# ========================================
# 11) Search spaces
#     - Stage 1: broad (fresh run)
# ========================================

# Stage 1 (broad): good for a fresh, clean run
def search_space_stage1(trial, tune_epochs):
    return {
        "dropout_rate": trial.suggest_float("dropout_rate", 0.10, 0.35),
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0.00, 0.10),
        "warmup_ratio": trial.suggest_float("warmup_ratio", 0.00, 0.12),
        "lr_scheduler_type": trial.suggest_categorical("lr_scheduler_type",
                            ["linear", "cosine", "cosine_with_restarts", "polynomial"]),
        "label_smoothing_factor": trial.suggest_float("label_smoothing_factor", 0.00, 0.08),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32]),
        "gradient_accumulation_steps": trial.suggest_categorical("gradient_accumulation_steps", [1, 2]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", tune_epochs[0], tune_epochs[1]),
    }

In [ ]:
# ========================================
# 12) Fresh run (Stage 1): tune → final → save weights to Drive
#     - Change N_TRIALS if you want more/less
# ========================================
N_TRIALS = 12
best_params_s1, best_val_s1 = tune_once(
    config_name="stage1_broad",
    search_space_fn=search_space_stage1,
    n_trials=N_TRIALS,
    tune_epochs=(4,6)
)

save_path_s1, val_metrics_s1, test_metrics_s1 = final_train_and_save(
    config_name="stage1_broad",
    best_params=best_params_s1,
    final_epochs=12,
    save_name="HF_best_model_stage1.pt"  # or any name you like
)

print("Stage 1 — Validation:", val_metrics_s1)
print("Stage 1 — Test:", test_metrics_s1)


[I 2025-08-09 08:01:04,582] A new study created in memory with name: stage1_broad_20250809_080104


  0%|          | 0/12 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,F1 Weighted
1,0.542300,0.485204,0.822157,0.822157,0.828923,0.822927
2,0.364300,0.432410,0.833333,0.833333,0.837068,0.833794
3,0.250500,0.395597,0.857993,0.857993,0.863109,0.857918
4,0.162300,0.447325,0.870991,0.870991,0.874817,0.870774
5,0.127600,0.480414,0.874393,0.874393,0.877225,0.874078


[I 2025-08-09 09:29:38,984] Trial 11 finished with value: 0.8740781047064323 and parameters: {'dropout_rate': 0.25698452656069404, 'learning_rate': 4.90971587490841e-05, 'weight_decay': 0.09946462260260763, 'warmup_ratio': 0.03169401711989029, 'lr_scheduler_type': 'cosine', 'label_smoothing_factor': 0.07746766398093248, 'per_device_train_batch_size': 16, 'gradient_accumulation_steps': 2, 'num_train_epochs': 5}. Best is trial 3 with value: 0.8762058535274788.
======= Tuning finished: stage1_broad | best f1_weighted=0.8762


Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,F1 Weighted
1,0.672200,0.748919,0.679179,0.679179,0.687518,0.673857
2,0.456600,0.481502,0.818027,0.818027,0.822647,0.819106
3,0.324000,0.392218,0.864553,0.864553,0.868755,0.864760
4,0.294100,0.561442,0.842809,0.842809,0.848362,0.842666
5,0.221800,0.522227,0.867104,0.867104,0.868482,0.867189
6,0.187100,0.556427,0.872449,0.872449,0.874969,0.872221
7,0.165300,0.580276,0.868440,0.868440,0.871292,0.868070
8,0.129100,0.714697,0.871963,0.871963,0.874618,0.872159


========= Final best weights saved to: /content/drive/MyDrive/deep_learning/HF_best_model_stage1.pt
Stage 1 — Validation: {'eval_loss': 0.7146971821784973, 'eval_accuracy': 0.8719630709426628, 'eval_f1_micro': 0.8719630709426628, 'eval_f1_macro': 0.8746176124361495, 'eval_f1_weighted': 0.8721587403618362, 'eval_runtime': 2.4239, 'eval_samples_per_second': 3396.245, 'eval_steps_per_second': 53.221, 'epoch': 8.0}
Stage 1 — Test: {'eval_loss': 0.8786630630493164, 'eval_accuracy': 0.8430753027909426, 'eval_f1_micro': 0.8430753027909426, 'eval_f1_macro': 0.8468222358404913, 'eval_f1_weighted': 0.843127454741659, 'eval_runtime': 1.3459, 'eval_samples_per_second': 2821.871, 'eval_steps_per_second': 44.579, 'epoch': 8.0}


In [14]:

# Stage 2 (narrow): refine around a known good region (optional)
def search_space_stage2(trial, tune_epochs):
    return {
        "dropout_rate": trial.suggest_float("dropout_rate", 0.20, 0.30),
        "learning_rate": trial.suggest_float("learning_rate", 3.5e-5, 6.5e-5, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0.036, 0.076),
        "warmup_ratio": trial.suggest_float("warmup_ratio", 0.04, 0.10),
        "lr_scheduler_type": trial.suggest_categorical("lr_scheduler_type", ["linear", "polynomial"]),
        "label_smoothing_factor": trial.suggest_float("label_smoothing_factor", 0.03, 0.07),
        "per_device_train_batch_size": 16,
        "gradient_accumulation_steps": 2,
        "num_train_epochs": trial.suggest_int("num_train_epochs", tune_epochs[0], tune_epochs[1]),
    }


In [15]:
# ========================================
# 12) Fresh run (Stage 2): tune → final → save weights to Drive
#     - Change N_TRIALS if you want more/less
# ========================================
N_TRIALS = 12
best_params_s2, best_val_s2 = tune_once(
    config_name="stage2_Focused",
    search_space_fn=search_space_stage2,
    n_trials=N_TRIALS,
    tune_epochs=(4,6)
)

save_path_s2, val_metrics_s2, test_metrics_s2 = final_train_and_save(
    config_name="stage2_Focused",
    best_params=best_params_s2,
    final_epochs=12,
    save_name="HF_best_model_stage2.pt"  # or any name you like
)

print("Stage 2 — Validation:", val_metrics_s2)
print("Stage 2 — Test:", test_metrics_s2)


[I 2025-08-09 15:15:44,320] A new study created in memory with name: stage2_Focused_20250809_151544


  0%|          | 0/12 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,F1 Weighted
1,0.574600,0.534362,0.798348,0.798348,0.805857,0.798054
2,0.396300,0.446483,0.833698,0.833698,0.837968,0.833889
3,0.256700,0.437208,0.851555,0.851555,0.856839,0.851975
4,0.173100,0.373476,0.879009,0.879009,0.881282,0.878971
5,0.118500,0.463796,0.885326,0.885326,0.886935,0.885255


[I 2025-08-09 16:51:23,833] Trial 11 finished with value: 0.8852545363051838 and parameters: {'dropout_rate': 0.2991565721949841, 'learning_rate': 6.393134459385403e-05, 'weight_decay': 0.07575768494377597, 'warmup_ratio': 0.0998559427737265, 'lr_scheduler_type': 'polynomial', 'label_smoothing_factor': 0.03153000622859889, 'num_train_epochs': 5}. Best is trial 11 with value: 0.8852545363051838.
======= Tuning finished: stage2_Focused | best f1_weighted=0.8853


Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,F1 Weighted
1,0.644500,0.617834,0.733115,0.733115,0.743275,0.730380
2,0.476000,0.455825,0.827745,0.827745,0.833122,0.828513
3,0.359300,0.409282,0.847546,0.847546,0.851947,0.847526
4,0.269600,0.388543,0.851919,0.851919,0.856555,0.851689
5,0.227900,0.463937,0.861516,0.861516,0.865422,0.861641
6,0.200100,0.549734,0.862609,0.862609,0.866046,0.862786
7,0.154500,0.589663,0.856293,0.856293,0.859279,0.856296
8,0.100100,0.523143,0.877794,0.877794,0.880400,0.877702
9,0.064200,0.697707,0.859694,0.859694,0.862211,0.859346
10,0.048100,0.767870,0.870991,0.870991,0.872140,0.871016


========= Final best weights saved to: /content/drive/MyDrive/deep_learning/HF_best_model_stage2.pt
Stage 2 — Validation: {'eval_loss': 0.7678701877593994, 'eval_accuracy': 0.8709912536443148, 'eval_f1_micro': 0.8709912536443148, 'eval_f1_macro': 0.8721400933619172, 'eval_f1_weighted': 0.8710157188065676, 'eval_runtime': 2.3628, 'eval_samples_per_second': 3483.956, 'eval_steps_per_second': 54.596, 'epoch': 10.0}
Stage 2 — Test: {'eval_loss': 0.8851942420005798, 'eval_accuracy': 0.8454449710373881, 'eval_f1_micro': 0.8454449710373881, 'eval_f1_macro': 0.8484664652016841, 'eval_f1_weighted': 0.8462894777579669, 'eval_runtime': 1.251, 'eval_samples_per_second': 3036.046, 'eval_steps_per_second': 47.963, 'epoch': 10.0}


In [20]:

# Stage 3 (narrow + dirty data):
def search_space_stage2(trial, tune_epochs):
    return {
        "dropout_rate": trial.suggest_float("dropout_rate", 0.20, 0.30),
        "learning_rate": trial.suggest_float("learning_rate", 3.5e-5, 6.5e-5, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0.036, 0.076),
        "warmup_ratio": trial.suggest_float("warmup_ratio", 0.04, 0.10),
        "lr_scheduler_type": trial.suggest_categorical("lr_scheduler_type", ["linear", "polynomial"]),
        "label_smoothing_factor": trial.suggest_float("label_smoothing_factor", 0.03, 0.07),
        "per_device_train_batch_size": 16,
        "gradient_accumulation_steps": 2,
        "num_train_epochs": trial.suggest_int("num_train_epochs", tune_epochs[0], tune_epochs[1]),
    }


In [21]:
# ========================================
# 13) Fresh run (Stage 3): tune → final → save weights to Drive
#     - Change N_TRIALS if you want more/less
# ========================================
N_TRIALS = 12
best_params_s3, best_val_s3 = tune_once(
    config_name="stage3_dirty",
    search_space_fn=search_space_stage2,
    n_trials=N_TRIALS,
    tune_epochs=(4,6)
)



[I 2025-08-10 09:36:12,058] A new study created in memory with name: stage3_dirty_20250810_093612


  0%|          | 0/12 [00:00<?, ?it/s]

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,F1 Weighted
1,0.567300,0.551537,0.772473,0.772473,0.781631,0.770432
2,0.416900,0.425214,0.848154,0.848154,0.852314,0.848682
3,0.265000,0.385716,0.866861,0.866861,0.871344,0.866842
4,0.149200,0.441417,0.869048,0.869048,0.872856,0.869152
5,0.105800,0.535247,0.875000,0.875000,0.877639,0.874668


[I 2025-08-10 09:44:12,992] Trial 0 finished with value: 0.8746682112893818 and parameters: {'dropout_rate': 0.24913335524808675, 'learning_rate': 5.16129024995116e-05, 'weight_decay': 0.04726412273963576, 'warmup_ratio': 0.09864833355085822, 'lr_scheduler_type': 'polynomial', 'label_smoothing_factor': 0.057029497665855855, 'num_train_epochs': 5}. Best is trial 0 with value: 0.8746682112893818.


...

[I 2025-08-10 11:18:01,751] Trial 11 finished with value: 0.8768979422069226 and parameters: {'dropout_rate': 0.2754645660175671, 'learning_rate': 6.268838598045621e-05, 'weight_decay': 0.06276783970761148, 'warmup_ratio': 0.04050797856461872, 'lr_scheduler_type': 'linear', 'label_smoothing_factor': 0.03068289131102686, 'num_train_epochs': 6}. Best is trial 5 with value: 0.8783334539034551.
======= Tuning finished: stage3_dirty | best f1_weighted=0.8783


Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,F1 Weighted
1,0.643400,0.634048,0.744655,0.744655,0.747999,0.742057
2,0.461000,0.515569,0.806973,0.806973,0.806858,0.804676
3,0.293200,0.402411,0.853134,0.853134,0.857113,0.852900
4,0.208700,0.477818,0.848275,0.848275,0.853023,0.847770
5,0.179500,0.488750,0.850826,0.850826,0.854971,0.850443


========= Final best weights saved to: /content/drive/MyDrive/deep_learning/HF_best_model_stage3.pt


NameError: name 'val_metrics_s2' is not defined

In [23]:
save_path_s3, val_metrics_s3, test_metrics_s3 = final_train_and_save(
    config_name="stage3_dirty",
    best_params=best_params_s3,
    final_epochs=12,
    save_name="HF_best_model_stage3.pt"  # or any name you like
)

print("Stage 3 — Validation:", val_metrics_s3)
print("Stage 3 — Test:", test_metrics_s3)


Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,F1 Weighted
1,0.613100,0.614851,0.769922,0.769922,0.777231,0.769656
2,0.426800,0.484976,0.813897,0.813897,0.817449,0.814118
3,0.306500,0.439067,0.835155,0.835155,0.839209,0.834135
4,0.219000,0.442670,0.850705,0.850705,0.854587,0.850782
5,0.196400,0.526224,0.862488,0.862488,0.865026,0.862622
6,0.170800,0.525297,0.867104,0.867104,0.870411,0.867081
7,0.110300,0.556381,0.866375,0.866375,0.869620,0.866328
8,0.076000,0.680139,0.866254,0.866254,0.869698,0.866096


========= Final best weights saved to: /content/drive/MyDrive/deep_learning/HF_best_model_stage3.pt
Stage 3 — Validation: {'eval_loss': 0.680138885974884, 'eval_accuracy': 0.8662536443148688, 'eval_f1_micro': 0.8662536443148688, 'eval_f1_macro': 0.869698085443243, 'eval_f1_weighted': 0.8660961084936232, 'eval_runtime': 2.3698, 'eval_samples_per_second': 3473.691, 'eval_steps_per_second': 54.435, 'epoch': 8.0}
Stage 3 — Test: {'eval_loss': 0.7757037878036499, 'eval_accuracy': 0.8438651922064244, 'eval_f1_micro': 0.8438651922064244, 'eval_f1_macro': 0.8473504043378307, 'eval_f1_weighted': 0.8436194392822652, 'eval_runtime': 1.2833, 'eval_samples_per_second': 2959.585, 'eval_steps_per_second': 46.755, 'epoch': 8.0}
